In [1]:
from Bio import Phylo
from Bio import *
from dendropy import Tree
import shutil
import os



<h3 align="center"> <b>Limpeza</b> </h3>


In [2]:
def clean_dir(path_clean):
    if(os.path.exists(path_clean)):
        for name in os.listdir(path_clean):
            if(name != "file.gitkeep"):
                file_path = os.path.join(path_clean, name)
                os.remove(file_path)
        shutil.rmtree(path_clean)


def clean_files():
    dir_tmp = '../data/out/Subtrees/' 
    arquivos_tmp = os.listdir(dir_tmp)

    for name_file in arquivos_tmp:
        path_tmp = dir_tmp + "{name}".format(name = name_file)
        if name_file != "file.gitkeep":
            os.remove(path_tmp)
clean_files()



<h3 align="center"> <b>Print subtree</b> </h3>


In [3]:
def print_trees_in_directory(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".nexus"):
            filepath = os.path.join(directory, filename)
            
            print(filename.upper() + "\n")
            tree = Tree.get_from_path(filepath, "nexus")
            tree.print_plot()




<h3 align="center"> <b>Construtor subtree</b> </h3>


In [4]:
def sub_tree(path, name_subtree):
    tree = Phylo.read(path, "nexus")
    name_subtree = name_subtree.rsplit(".", 1)[0]#spit reverso

    #Lista com caminhos das subarvores, sera usada para compo a matriz de subarvores
    row_subtree = []

    # Iterar sobre todas as subárvores e salvar em arquivos separados
    for clade in tree.find_clades():
        subtree = Phylo.BaseTree.Tree(clade)
        if subtree.count_terminals() > 1:
            # Salva subarvore com nome
            
            filepath_out = "../data/out/Subtrees/{}_{}.nexus".format(name_subtree,clade.name)
            Phylo.write(subtree, filepath_out, "nexus")        
            row_subtree.append(filepath_out)
            
    return row_subtree
            
    
def sub_tree_separate_in_folder(path, name_subtree):
    name_subtree = name_subtree.rsplit(".", 1)[0]#spit reverso
    if(not(os.path.exists("../data/out/Subtrees/{}".format(name_subtree)))): 
        os.mkdir("../data/out/Subtrees/{}".format(name_subtree))#cria a pasta para salvar as sub da arv especifica
        
    tree = Phylo.read(path, "nexus")
    
    #Lista com caminhos das subarvores, sera usada para compo a matriz de subarvores
    row_subtree = []

    # Iterar sobre todas as subárvores e salvar em arquivos separados
    for clade in tree.find_clades():
        subtree = Phylo.BaseTree.Tree(clade)
        if subtree.count_terminals() > 1:
            # Salva subarvore com nome
            filepath_out = "../data/out/Subtrees/{}/sub{}_{}.nexus".format(name_subtree,name_subtree,clade.name)
            Phylo.write(subtree, filepath_out, "nexus") 
            row_subtree.append(filepath_out)
    return row_subtree
        

In [5]:
def directory_has_single_file(directory_path):
    if not os.path.isdir(directory_path):
        return False

    files = os.listdir(directory_path)
    if len(files) != 1:
        return False

    return True

<h3 align="center"> <b>Chamada funçoes</b> </h3>


<p align="center"> <b> ATENÇAO SE A ARVORE DE ENTRADA NAO FOR NEXUS MUDAR O SCHEMA EM Phylo.write( )
</b> </p>


In [6]:
# Diretório de input de arvores 
dir = '../data/out/Trees/' 
arquivos = os.listdir(dir)

# dir de saída
dir_path_out = "../data/out/Subtrees/"

#matriz com todas as sub arvores
matriz_subtree = []

for name_file in arquivos:
    if(name_file != "file.gitkeep"):

        dir_path_out_epsecif = "../data/out/Subtrees/{}".format(name_file.split(".")[0])
        
        # Gera as subarvores para todas as arvores na pasta /Trees salvando em arquivos
        matriz_subtree.append(sub_tree("{}{}".format(dir, name_file), name_file))
        
        # Gera as subarvores para todas as arvores na pasta /Trees salvando em pastas
        # matriz_subtree.append(sub_tree_separate_in_folder("{}{}".format(dir, name_file), name_file))
        
        # Limpa diritorio atual ( exclui pastas e arquivos)
        # clean_dir(dir_path_out_epsecif)

# Limpa diretorio atual ( exclui somente arquivos )
# clean_files()


In [7]:
# Encontrar o número máximo de colunas
max_colunas = max(len(row) for row in matriz_subtree)
# Encontrar o número máximo de linhas
max_linhas = len(matriz_subtree)

def preencher_matriz(matriz, valor_preenchimento):
    # Preencher as células vazias com o valor de preenchimento
    for row in matriz:
        while len(row) < max_colunas:
            row.append(valor_preenchimento)

    return matriz


print(max_linhas, max_colunas)

8 8


In [8]:
#sobreescreve matriz igualando numero de colunas ( preenche com None celulás vazias )
matriz_subtree = preencher_matriz(matriz_subtree, None)

for linha in matriz_subtree:
    print(linha)


['../data/out/Subtrees/tree_ORTHOMCL1_Inner3.nexus', '../data/out/Subtrees/tree_ORTHOMCL1_Inner2.nexus', '../data/out/Subtrees/tree_ORTHOMCL1_Inner1.nexus', None, None, None, None, None]
['../data/out/Subtrees/tree_ORTHOMCL256_Inner8.nexus', '../data/out/Subtrees/tree_ORTHOMCL256_Inner7.nexus', '../data/out/Subtrees/tree_ORTHOMCL256_Inner6.nexus', '../data/out/Subtrees/tree_ORTHOMCL256_Inner5.nexus', '../data/out/Subtrees/tree_ORTHOMCL256_Inner3.nexus', '../data/out/Subtrees/tree_ORTHOMCL256_Inner2.nexus', '../data/out/Subtrees/tree_ORTHOMCL256_Inner1.nexus', '../data/out/Subtrees/tree_ORTHOMCL256_Inner4.nexus']
['../data/out/Subtrees/tree_ORTHOMCL256_2_Inner8.nexus', '../data/out/Subtrees/tree_ORTHOMCL256_2_Inner7.nexus', '../data/out/Subtrees/tree_ORTHOMCL256_2_Inner6.nexus', '../data/out/Subtrees/tree_ORTHOMCL256_2_Inner5.nexus', '../data/out/Subtrees/tree_ORTHOMCL256_2_Inner3.nexus', '../data/out/Subtrees/tree_ORTHOMCL256_2_Inner2.nexus', '../data/out/Subtrees/tree_ORTHOMCL256_2_In

<h3 align="center"> <b> Comparação das subarvores
</b> </h3>


In [9]:


def grau_MAF(path_1, path_2):
    if(path_1 is None or path_2 is None):
        return -1
        
    grau = 0

    subtree_1 = Phylo.read(path_1,'nexus')
    subtree_2 = Phylo.read(path_2,'nexus')

    # Lista todas as clades ( folhas )
    list_1 = [i.name for i in subtree_1.get_terminals()]
    list_2 = [i.name for i in subtree_2.get_terminals()]

    sorted_list1 = sorted(list_1)
    sorted_list2 = sorted(list_2)
    
    for i in range(len(list_1)):
        for j in range(len(list_2)):
            if sorted_list1[i] == sorted_list2[j]:
                grau += 1
    return grau


In [10]:
dict_maf_database = {}

def preencher_dict(dict, max_colunas):
    for i in range(max_colunas):
        dict[i+1] = {}

    return dict_maf_database

dict_maf_database = preencher_dict(dict_maf_database,max_colunas)
print(dict_maf_database)

{1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}, 8: {}}


In [11]:
# dict_maf_database = {}
max_maf = 0
for i in range(max_linhas):
    for j in range(max_colunas):
        dict_aux = {}
        for k in range(max_linhas):
            for l in range(max_colunas): 
                if i != k:
                    if max_maf <= grau_MAF(matriz_subtree[i][j],matriz_subtree[k][l]):
                        max_maf = grau_MAF(matriz_subtree[i][j],matriz_subtree[k][l])

                    g_maf = grau_MAF(matriz_subtree[i][j], matriz_subtree[k][l])
                    if g_maf is not False and g_maf >= 1:# Comparaçao pode ser feita
                        if g_maf not in dict_maf_database:# Nao sobrescreve caso já exista
                            dict_maf_database[g_maf] = {}
                        if matriz_subtree[i][j] not in dict_maf_database[g_maf]:# Nao sobrescreve caso já exista
                            dict_maf_database[g_maf][matriz_subtree[i][j]] = []
                        dict_maf_database[g_maf][matriz_subtree[i][j]].append(matriz_subtree[k][l])


In [12]:
print(max_maf)


8


In [13]:
for i, j in dict_maf_database.items():
    for key, val in j.items():
        print(i, key, val)
    

1 ../data/out/Subtrees/tree_ORTHOMCL256_Inner8.nexus ['../data/out/Subtrees/tree_ORTHOMCL256_2_Inner4.nexus']
1 ../data/out/Subtrees/tree_ORTHOMCL256_Inner7.nexus ['../data/out/Subtrees/tree_ORTHOMCL256_2_Inner4.nexus']
1 ../data/out/Subtrees/tree_ORTHOMCL256_Inner6.nexus ['../data/out/Subtrees/tree_ORTHOMCL256_2_Inner4.nexus']
1 ../data/out/Subtrees/tree_ORTHOMCL256_Inner5.nexus ['../data/out/Subtrees/tree_ORTHOMCL256_2_Inner4.nexus']
1 ../data/out/Subtrees/tree_ORTHOMCL256_Inner4.nexus ['../data/out/Subtrees/tree_ORTHOMCL256_2_Inner8.nexus', '../data/out/Subtrees/tree_ORTHOMCL256_2_Inner7.nexus', '../data/out/Subtrees/tree_ORTHOMCL256_2_Inner6.nexus', '../data/out/Subtrees/tree_ORTHOMCL256_2_Inner5.nexus', '../data/out/Subtrees/tree_ORTHOMCL256_2_Inner4.nexus']
1 ../data/out/Subtrees/tree_ORTHOMCL256_2_Inner8.nexus ['../data/out/Subtrees/tree_ORTHOMCL256_Inner4.nexus']
1 ../data/out/Subtrees/tree_ORTHOMCL256_2_Inner7.nexus ['../data/out/Subtrees/tree_ORTHOMCL256_Inner4.nexus']
1 ../d